In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

In [ ]:
(dataset_train, dataset_test), dataset_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
dataset_info

In [ ]:
for x in dataset_train:
    img, label = x
    plt.imshow(img[:,:,0], cmap='gray')

    print("Each element in our dataset is of type:", type(x))
    print("The label of this element is:", label)
    print("The shape of this image is:", img.shape)

    break

In [ ]:
def normalize_img(image, label):
    """Normalizes images from unsigned 8-bit integers to 32-bit float.
    Also scales all values to be between 0 and 1 rather than 0 and 255."""
    image = tf.cast(image, tf.float32)
    image = image / 255

    return image, label

dataset_train = dataset_train.map(normalize_img)

dataset_train = dataset_train.shuffle(dataset_info.splits['train'].num_examples)

dataset_train = dataset_train.batch(128)

dataset_test = dataset_test.map(normalize_img)
dataset_test = dataset_test.batch(128)

In [ ]:

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(10),
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
results = model.evaluate(dataset_train)
print("Our training loss and accuracy:", results)

results = model.evaluate(dataset_test)
print("Our testing loss and accuracy:", results)

In [ ]:
history = model.fit(
    dataset_train,
    epochs=8,
    validation_data=dataset_test,
)

Can you guess why the accuracy from our training dataset is high than our testing dataset accuracy?

In [ ]:
results = model.evaluate(dataset_train)
print("Our training loss and accuracy:", results)

results = model.evaluate(dataset_test)
print("Our testing loss and accuracy:", results)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')
plt.show()

In [ ]:
dataset_test = dataset_test.unbatch()
dataset_test = dataset_test.shuffle(buffer_size=dataset_info.splits['test'].num_examples)

In [ ]:
def random_prediction():
    img, label = list(dataset_test.take(1))[0]
    img = tf.reshape(img, (-1, 28, 28, 1))
    print(img.shape, label.shape)

    prediction = model.predict(img)
    print("This is the prediction without it being processed first:\n", prediction)

    processed_prediction = tf.math.argmax(prediction[0])

    print("\nThe model's prediction for the image:", int(processed_prediction))
    print("The actual label:", int(label))
    plt.imshow(img[0,:,:,0], cmap="gray")


random_prediction()